### MID 7
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### SVD used for feature reduction, n_components = 15.
### Oversampling and smote methods used to increase size of training data.
###  Deep neural network used, model type is 3 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
import tensorflow as tf
import csv
from sklearn.decomposition import TruncatedSVD

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [4]:
combined_data

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,0.142857,0.6,0.050,0.102941,0.214286,0.011856,0.014815,...,1.000000,0.111111,0.885057,0.232323,0.75,0.363636,0.165951,0.085714,0.209085,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,0.181818,0.0,0.025,0.058824,0.163265,0.020033,0.029630,...,1.000000,0.000000,0.000000,0.307692,0.00,0.000000,0.080979,0.171429,0.269903,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,0.038961,0.2,0.000,0.007353,0.045918,0.000000,0.000000,...,1.000000,0.111111,0.714286,0.666667,0.25,0.090909,0.083267,0.028571,0.109529,0
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,0.155844,0.4,0.000,0.088235,0.188776,0.013083,0.000000,...,1.000000,0.111111,0.770833,0.458333,0.00,0.000000,0.115693,0.085714,0.232742,0
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,0.077922,0.4,0.000,0.029412,0.107143,0.000409,0.000000,...,1.000000,0.000000,0.880952,0.416667,0.50,0.181818,0.100881,0.028571,0.136898,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.ant.TaskAdapter,0.064935,0.4,0.000,0.051471,0.096939,0.001635,0.029630,...,0.000000,0.000000,0.902439,0.400000,0.25,0.090909,0.138351,0.028571,0.131428,0
174,ant,1.4,org.apache.tools.ant.taskdefs.rmic.DefaultRmic...,0.220779,0.0,0.075,0.110294,0.336735,0.036795,0.029630,...,1.000000,0.222222,0.000000,0.197917,0.00,0.000000,0.172373,0.400000,0.396221,0
175,ant,1.4,org.apache.tools.ant.IntrospectionHelper,0.298701,0.0,0.000,0.227941,0.418367,0.082175,0.162963,...,0.444444,0.000000,0.000000,0.318182,0.00,0.000000,0.219724,0.685714,0.378561,0
176,ant,1.4,org.apache.tools.ant.taskdefs.compilers.Defaul...,0.142857,0.0,0.200,0.161765,0.438776,0.006132,0.059259,...,1.000000,0.777778,0.000000,0.266667,0.00,0.000000,0.373390,0.628571,0.761689,1


In [5]:
# define transform
components = 15
svd = TruncatedSVD(n_components=components)
# prepare transform on dataset
svd.fit(combined_data[cols_to_norm])
# apply transform to dataset
transformed = svd.transform(combined_data[cols_to_norm])

In [6]:
transformed = pd.DataFrame(transformed)
transformed 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.697270,-0.240716,0.571264,0.167873,-0.202270,-0.188764,-0.085932,0.036019,-0.075786,0.129869,-0.074938,-0.025339,-0.066285,0.001433,-0.015661
1,1.016611,-0.267720,-0.587890,-0.054598,-0.014350,-0.077750,-0.160155,0.052980,0.100634,-0.030916,-0.066994,-0.048365,-0.007133,0.003084,0.027329
2,1.298451,-0.173527,0.139837,-0.541003,-0.007964,0.011583,0.156168,-0.110641,-0.028186,0.008754,0.139273,-0.107446,-0.018334,0.006414,-0.066103
3,1.537891,-0.056428,0.028502,-0.104266,0.280907,0.130204,-0.176106,-0.029262,-0.088801,-0.025663,-0.118818,0.003933,0.018829,-0.043154,0.048522
4,1.530257,-0.074334,0.374525,-0.205480,-0.061914,-0.152907,-0.063597,0.021766,0.089365,0.037752,0.031946,-0.114260,-0.091288,-0.014637,-0.040312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.886501,0.524176,0.551332,0.103135,0.083555,0.172359,0.095620,0.012480,0.004141,0.017085,0.037964,-0.029034,-0.045604,-0.017821,0.022297
299,1.128822,-0.426056,-0.629426,0.258850,-0.034627,0.177394,-0.220035,0.025682,-0.128906,0.061409,-0.099610,-0.022340,0.050520,0.067642,-0.067599
300,1.027511,-0.126484,-0.591917,0.769665,0.027848,0.483424,0.049615,-0.134999,-0.142810,0.233455,-0.574456,-0.054281,0.160251,0.095574,-0.128321
301,1.318002,-0.514244,-0.775534,0.478137,-0.191742,0.531927,-0.176518,0.103397,-0.589727,0.040915,0.212815,0.047857,0.022727,0.095875,-0.128328


In [7]:
train_data_index_list = list(range(ant_1_3.shape[0]))
train_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124]

In [8]:
test_data_index_list = list(range(ant_1_3.shape[0], ant_1_3.shape[0] + ant_1_4.shape[0]))
test_data_index_list

[125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291

In [9]:
X_train = transformed[transformed.index.isin(train_data_index_list)]
X_train = np.array(X_train)

X_test = transformed[transformed.index.isin(test_data_index_list)]
X_test = np.array(X_test)

In [10]:
Y_train = combined_data[transformed.index.isin(train_data_index_list)]
Y_train = Y_train['bug']

In [11]:
Y_test = combined_data[transformed.index.isin(test_data_index_list)]
Y_test = Y_test['bug']

In [12]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [13]:
train_x

array([[[ 1.69726979e+00, -2.40716233e-01,  5.71263952e-01, ...,
         -6.62849517e-02,  1.43327120e-03, -1.56612043e-02]],

       [[ 1.01661138e+00, -2.67720025e-01, -5.87889971e-01, ...,
         -7.13327593e-03,  3.08443645e-03,  2.73289410e-02]],

       [[ 1.29845060e+00, -1.73526753e-01,  1.39837075e-01, ...,
         -1.83338282e-02,  6.41379625e-03, -6.61028019e-02]],

       ...,

       [[ 1.01420045e+00, -3.74213023e-01, -5.47215559e-01, ...,
          3.08422923e-02, -4.45360308e-02, -9.61714293e-03]],

       [[ 1.01420045e+00, -3.74213023e-01, -5.47215559e-01, ...,
          3.08422923e-02, -4.45360308e-02, -9.61714293e-03]],

       [[ 1.53857934e+00, -2.25167246e-01,  4.45008530e-01, ...,
          6.04008611e-02, -5.63934797e-02, -1.12370553e-03]]])

In [14]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [15]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [16]:
test_x

array([[[ 1.52329275, -0.17009031,  0.12845599, ..., -0.03488069,
         -0.02853661, -0.02754137]],

       [[ 1.65167769, -0.06325479,  0.36856482, ...,  0.00259279,
         -0.02678534,  0.01689497]],

       [[ 1.02228015, -0.14704317, -0.59470571, ...,  0.00630524,
          0.01799199,  0.02629242]],

       ...,

       [[ 1.02751126, -0.1264845 , -0.5919174 , ...,  0.16025114,
          0.095574  , -0.12832104]],

       [[ 1.31800237, -0.51424357, -0.77553422, ...,  0.02272707,
          0.09587535, -0.12832784]],

       [[ 1.53449933,  0.03236187,  0.0688109 , ..., -0.01911218,
          0.00320715, -0.03528819]]])

In [17]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [18]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [19]:
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(100, input_shape = (1,components), dropout = 0.2, return_sequences=True))
model.add(LSTM(80, dropout = 0.2, return_sequences=True))
model.add(LSTM(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
# Designing and initializing the model.
model = Sequential()
model.add(Dense(15, activation = 'relu', input_dim = components))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
# Adding the output layer
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                240       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 796us/step - loss: 3.3469 - mse: 3.3469 - mae: 1.4750 - root_mean_squared_error: 1.8295 - mean_squared_logarithmic_error: 0.8261
Epoch 2/100
4/4 [==============================] - 0s 770us/step - loss: 3.2839 - mse: 3.2839 - mae: 1.4637 - root_mean_squared_error: 1.8122 - mean_squared_logarithmic_error: 0.7962
Epoch 3/100
4/4 [==============================] - 0s 1ms/step - loss: 3.2223 - mse: 3.2223 - mae: 1.4530 - root_mean_squared_error: 1.7951 - mean_squared_logarithmic_error: 0.7682
Epoch 4/100
4/4 [==============================] - 0s 682us/step - loss: 3.1596 - mse: 3.1596 - mae: 1.4419 - root_mean_squared_error: 1.7775 - mean_squared_logarithmic_error: 0.7405
Epoch 5/100
4/4 [==============================] - 0s 1ms/step - loss: 3.0974 - mse: 3.0974 - mae: 1.4303 - root_mean_squared_error: 1.7599 - mean_squared_logarithmic_error: 0.7136
Epoch 6/100
4/4 [==============================] - 0s 1ms/step - loss: 3.0326 - mse: 3.03

4/4 [==============================] - 0s 664us/step - loss: 0.6182 - mse: 0.6182 - mae: 0.6520 - root_mean_squared_error: 0.7863 - mean_squared_logarithmic_error: 0.1595
Epoch 88/100
4/4 [==============================] - 0s 926us/step - loss: 0.6109 - mse: 0.6109 - mae: 0.6489 - root_mean_squared_error: 0.7816 - mean_squared_logarithmic_error: 0.1570
Epoch 89/100
4/4 [==============================] - 0s 698us/step - loss: 0.6034 - mse: 0.6034 - mae: 0.6449 - root_mean_squared_error: 0.7768 - mean_squared_logarithmic_error: 0.1549
Epoch 90/100
4/4 [==============================] - 0s 983us/step - loss: 0.5957 - mse: 0.5957 - mae: 0.6395 - root_mean_squared_error: 0.7718 - mean_squared_logarithmic_error: 0.1532
Epoch 91/100
4/4 [==============================] - 0s 667us/step - loss: 0.5878 - mse: 0.5878 - mae: 0.6345 - root_mean_squared_error: 0.7667 - mean_squared_logarithmic_error: 0.1511
Epoch 92/100
4/4 [==============================] - 0s 675us/step - loss: 0.5804 - mse: 0.580

In [24]:
# Getting the predictions.
predictions_y = model.predict(test_x)
predictions_y

array([[[1.1334407 ]],

       [[1.7524495 ]],

       [[0.9087959 ]],

       [[1.4855871 ]],

       [[1.6402361 ]],

       [[1.4108789 ]],

       [[1.5698905 ]],

       [[0.2353195 ]],

       [[0.6107452 ]],

       [[2.699305  ]],

       [[0.94542533]],

       [[2.2298372 ]],

       [[0.57242656]],

       [[1.4197145 ]],

       [[1.5444288 ]],

       [[0.2353195 ]],

       [[2.5671415 ]],

       [[0.33354235]],

       [[0.2353195 ]],

       [[0.7105511 ]],

       [[0.9231513 ]],

       [[0.72738063]],

       [[0.6096236 ]],

       [[1.2254436 ]],

       [[1.9677575 ]],

       [[1.3487101 ]],

       [[1.376359  ]],

       [[0.7878943 ]],

       [[1.2513418 ]],

       [[0.2353195 ]],

       [[0.2353195 ]],

       [[0.39049965]],

       [[0.36382502]],

       [[1.459171  ]],

       [[0.62933356]],

       [[0.2353195 ]],

       [[1.2060711 ]],

       [[0.68176246]],

       [[0.9900902 ]],

       [[0.23532864]],

       [[1.7340326 ]],

       [[0.23531

In [25]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[[1.]],

       [[2.]],

       [[1.]],

       [[1.]],

       [[2.]],

       [[1.]],

       [[2.]],

       [[0.]],

       [[1.]],

       [[3.]],

       [[1.]],

       [[2.]],

       [[1.]],

       [[1.]],

       [[2.]],

       [[0.]],

       [[3.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[2.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[2.]],

       [[0.]],

       [[0.]],

       [[2.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[2.]],

       [[3.]],

       [[1.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[0.]],

       [[2.]],

       [[1.]],

       [[0.]],

       [[0.]],

       [[1.]],

       [[2.]],

       [

In [26]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1, 1)

In [27]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([[169.]], dtype=float32)

In [28]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([[0.48361152]], dtype=float32)

In [29]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([[0.49200496]], dtype=float32)

In [30]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 581us/step - loss: 1.2139 - mse: 1.2139 - mae: 0.8938 - root_mean_squared_error: 1.1018 - mean_squared_logarithmic_error: 0.4127
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [31]:
# Saving the model
model_id = 7
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model3/ant1.3_ant1.4_model3_1'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model3/ant1.3_ant1.4_model3_1/assets


In [32]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [33]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [34]:
score

[7,
 'ant-1.3',
 'ant-1.4',
 1.2139216661453247,
 1.2139216661453247,
 0.893781840801239,
 1.1017811298370361,
 0.41268062591552734,
 0.483611524105072,
 0.49200496077537537]

In [35]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [36]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)